# Парсер новостных текстов с сайтов РБК и Лента.ру

## Импорт библиотек и описание классов

In [1]:
# Установка библиотек
!pip install bs4
!pip install openpyxl

  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=4cc297f2cb37d555da3978225372e0199185ffdf5b4bfbc4c3fd498c743adf6b
  Stored in directory: /home/misha-sh/.cache/pip/wheels/75/78/21/68b124549c9bdc94f822c02fb9aa3578a669843f9767776bca
Successfully built bs4
     |████████████████████████████████| 242 kB 2.1 MB/s eta 0:00:01


In [19]:
!pip3 install lxml

In [2]:
# Импорт библиотек
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from IPython import display

Ниже описаны классы для извлечения данных. 

Используются поисковые движки сайтов, возвращающие по запросу json таблицы с информацией о статьях. 

При их использовании есть различные ограничения, например, на количество статей в запросе. Так на сайте РБК выдается максимум 100 статей, в то время как на Ленте.ру можно получить сразу 1000. По моим наблюдениям, запросы на Лента.ру ограничены временем, и соответственно, точное ограничение найти не получится. Более того, на Лента.ру сразу выдается текст статьи в json таблице, с РБК приходится дополнительно парсить текст со страниц.  

### Парсер для РБК



In [2]:
class rbc_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями
        """
        url = 'https://www.rbc.ru/v10/search/ajax/?\
        project={0}&\
        category={1}&\
        dateFrom={2}&\
        dateTo={3}&\
        offset={4}&\
        limit={5}&\
        query={6}&\
        material={7}'.format(param_dict['project'],
                            param_dict['category'],
                            param_dict['dateFrom'],
                            param_dict['dateTo'],
                            param_dict['offset'],
                            param_dict['limit'],
                            param_dict['query'],
                            param_dict['material'])
        
        return url
    
    
    def _get_search_table(self, param_dict: dict,
                          includeText: bool = True) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        
        includeText: bool
        ### Если True, статьи возвращаются с текстами
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['items'])
        if includeText and not search_table.empty:
            get_text = lambda x: self._get_article_data(x['fronturl'])
            search_table[['overview', 'text']] = search_table.apply(get_text,
                                                                    axis=1).tolist()
            
        return search_table.sort_values('publish_date_t', ignore_index=True)
    
    
    def _get_article_data(self, url: str):
        """
        Возвращает описание и текст статьи по ссылке
        """
        r = rq.get(url)
        soup = bs(r.text, features="lxml") # features="lxml" чтобы не было warning
        div_overview = soup.find('div', {'class': 'article__text__overview'})
        if div_overview:
            overview = div_overview.text.replace('<br />','\n').strip()
        else:
            overview = None
        p_text = soup.find_all('p')
        if p_text:
            text = ' '.join(map(lambda x:
                                x.text.replace('<br />','\n').strip(),
                                p_text))
        else:
            text = None
        
        return overview, text 
    
    def get_articles(self,
                     param_dict,
                     time_step = 7,
                     save_every = 5,
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%d.%m.%Y')
        dateTo = datetime.strptime(param_copy['dateTo'], '%d.%m.%Y')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime("%d.%m.%Y")
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime("%d.%m.%Y")
            print('Parsing articles from ' + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime("%d.%m.%Y")
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0
        
        if save_excel:
            out.to_excel("rbc_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')
        
        return out

### Парсер для Лента.ру

In [3]:
class lentaRu_parser:
    def __init__(self):
        pass
    
    
    def _get_url(self, param_dict: dict) -> str:
        """
        Возвращает URL для запроса json таблицы со статьями

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from=0&'\                       # Смещение
        + 'size=1000&'\                    # Кол-во статей
        + 'sort=2&'\                       # Сортировка по дате (2), по релевантности (1)
        + 'title_only=0&'\                 # Точная фраза в заголовке
        + 'domain=1&'\                     # ??
        + 'modified%2Cformat=yyyy-MM-dd&'\ # Формат даты
        + 'type=1&'\                       # Материалы. Все материалы (0). Новость (1)
        + 'bloc=4&'\                       # Рубрика. Экономика (4). Все рубрики (0)
        + 'modified%2Cfrom=2020-01-01&'\
        + 'modified%2Cto=2020-11-01&'\
        + 'query='                         # Поисковой запрос
        """
        hasType = int(param_dict['type']) != 0
        hasBloc = int(param_dict['bloc']) != 0

        url = 'https://lenta.ru/search/v2/process?'\
        + 'from={}&'.format(param_dict['from'])\
        + 'size={}&'.format(param_dict['size'])\
        + 'sort={}&'.format(param_dict['sort'])\
        + 'title_only={}&'.format(param_dict['title_only'])\
        + 'domain={}&'.format(param_dict['domain'])\
        + 'modified%2Cformat=yyyy-MM-dd&'\
        + 'type={}&'.format(param_dict['type']) * hasType\
        + 'bloc={}&'.format(param_dict['bloc']) * hasBloc\
        + 'modified%2Cfrom={}&'.format(param_dict['dateFrom'])\
        + 'modified%2Cto={}&'.format(param_dict['dateTo'])\
        + 'query={}'.format(param_dict['query'])
        
        return url


    def _get_search_table(self, param_dict: dict) -> pd.DataFrame:
        """
        Возвращает pd.DataFrame со списком статей
        """
        url = self._get_url(param_dict)
        r = rq.get(url)
        search_table = pd.DataFrame(r.json()['matches'])
        
        return search_table

    
    def get_articles(self,
                     param_dict,
                     time_step = 37,
                     save_every = 5, 
                     save_excel = True) -> pd.DataFrame:
        """
        Функция для скачивания статей интервалами через каждые time_step дней
        Делает сохранение таблицы через каждые save_every * time_step дней

        param_dict: dict
        ### Параметры запроса 
        ###### project - раздел поиска, например, rbcnews
        ###### category - категория поиска, например, TopRbcRu_economics
        ###### dateFrom - с даты
        ###### dateTo - по дату
        ###### offset - смещение поисковой выдачи
        ###### limit - лимит статей, максимум 100
        ###### query - поисковой запрос (ключевое слово), например, РБК

        """
        param_copy = param_dict.copy()
        time_step = timedelta(days=time_step)
        dateFrom = datetime.strptime(param_copy['dateFrom'], '%Y-%m-%d')
        dateTo = datetime.strptime(param_copy['dateTo'], '%Y-%m-%d')
        if dateFrom > dateTo:
            raise ValueError('dateFrom should be less than dateTo')
        
        out = pd.DataFrame()
        save_counter = 0

        while dateFrom <= dateTo:
            param_copy['dateTo'] = (dateFrom + time_step).strftime('%Y-%m-%d')
            if dateFrom + time_step > dateTo:
                param_copy['dateTo'] = dateTo.strftime('%Y-%m-%d')
            print('Parsing articles from '\
                  + param_copy['dateFrom'] +  ' to ' + param_copy['dateTo'])
            out = out.append(self._get_search_table(param_copy), ignore_index=True)
            dateFrom += time_step + timedelta(days=1)
            param_copy['dateFrom'] = dateFrom.strftime('%Y-%m-%d')
            save_counter += 1
            if save_counter == save_every:
                display.clear_output(wait=True)
                out.to_excel("/tmp/checkpoint_table.xlsx")
                print('Checkpoint saved!')
                save_counter = 0
            
        if save_excel:
            out.to_excel("lenta_{}_{}.xlsx".format(
                param_dict['dateFrom'],
                param_dict['dateTo']))
        print('Finish')
        
        return out

## Пример выгрузки данных

### РБК

* __project__ - проекты РБК. Возможные значения: ["rbcnews", "rbctv", "rbcstyle", "sport", "realty", "crypto", "autonews", "quote", "bc3", "trends"]
 
* __category__ - рубрики: ["TopRbcRu_economics", "TopRbcRu_auto", "TopRbcRu_business", "TopRbcRu_money", "TopRbcRu_realty", "TopRbcRu_society", "TopRbcRu_politics", "TopRbcRu_own_business", "TopRbcRu_specials", "TopRbcRu_technology_and_media", "TopRbcRu_finances"]

* __material__ - материалы: ["video", "quiz", "interview", "research", "card", "opinion", "multimedia", "short_news", "olympics_online", "online", "investigation", "rating", "article_specproject", "article", "story"]

* __dateFrom__ - с даты 

* __dateTo__ - по дату 

* __offset__ - смещение поисковой выдачи (от 0 до 100)

* __limit__ - лимит запроса, максимум 100 

_Чтобы не специфировать параметр, оставляем поле пустым_

In [4]:
# Задаем параметры запросы и складываем в param_dict
use_parser = "РБК"

query = 'РБК'
project = "rbcnews"
category = "TopRbcRu_economics"
material = ""
dateFrom = '2021-01-01'
dateTo = "2021-02-28"
offset = 0
limit = 100

if use_parser == "РБК":
    param_dict = {'query'   : query, 
                  'project' : project,
                  'category': category,
                  'dateFrom': datetime.
                  strptime(dateFrom, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'dateTo'  : datetime.
                  strptime(dateTo, '%Y-%m-%d').
                  strftime('%d.%m.%Y'),
                  'offset'  : str(offset),
                  'limit'   : str(limit),
                  'material': material}

print(use_parser, "- param_dict:", param_dict)

РБК - param_dict: {'query': 'РБК', 'project': 'rbcnews', 'category': 'TopRbcRu_economics', 'dateFrom': '01.01.2021', 'dateTo': '28.02.2021', 'offset': '0', 'limit': '100', 'material': ''}


In [10]:
# Пример того, как выглядит json таблица запроса по параметрам.
# Действует ограничение в 100 статей на 1 запрос (параметром limit)
assert use_parser == "РБК"
parser = rbc_parser()
tbl = parser._get_search_table(param_dict,
                               includeText = True) # Парсить текст статей
print(len(tbl))
tbl.head()
tbl['text'][0]

10


'Нобелевский комитет, приняв решение о награждении премией правозащитного центра «Мемориал» (признан в России иноагентом и ликвидирован), сделал правильный выбор, заявила вдова первого президента России Наина Ельцина. Пресс-релиз с ее оценкой опубликован на сайте Ельцин-центра. «Сегодня стало известно, что Нобелевскую премию мира в числе других получил российский центр «Мемориал». Хотела\xa0бы от всего сердца поздравить с этой высокой наградой наших правозащитников. Для людей, работающих в «Мемориале», не было задачи важнее, чем защита конституционных прав и свобод людей, независимо от их политических убеждений, должностей или веры»,\xa0— отметила Ельцина. «Считаю, что Нобелевский комитет сделал очень правильный выбор. И дай Бог, чтобы правозащитники «Мемориала» и в будущем имели возможность работать на благо наших людей»,\xa0— подчеркнула она.  Кроме того, Ельцина напомнила,\xa0что «Мемориал» занимался разоблачением преступлений сталинского режима и считал это своей принципиальной зад

In [12]:
# Пример работы программы итеративного сбора большого количества текстов статей
# Работает, конечно, очень долго :(
table = parser.get_articles(param_dict=param_dict,
                             time_step = 7, # Шаг - 7 дней, можно больше,
                                            # но есть риск отсечения статей в неделях, гдестатей больше 100
                             save_every = 5, # Сохранять чекпойнт каждые 5 шагов
                             save_excel = True) # Сохранить итоговый файл
print(len(table))
table.head()

Checkpoint saved!
Parsing articles from 10.02.2021 to 17.02.2021


/tmp/ipykernel_5304/2151416731.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 18.02.2021 to 25.02.2021


/tmp/ipykernel_5304/2151416731.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 26.02.2021 to 28.02.2021
Finish
80


/tmp/ipykernel_5304/2151416731.py:105: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


,id,fronturl,publish_date_t,publish_date,title,photo,project,category,opinion_authors,authors,anons,overview,text
0,63404cc59a7947663eaa6bae,https://www.rbc.ru/politics/07/10/2022/63404cc...,1665158767,"Fri, 07 Oct 2022 19:06:07 +0300",Росстат перестал публиковать данные о смертнос...,{'url': 'https://s0.rbk.ru/v6_top_pics/resized...,None,Политика,None,"[Екатерина Виноградова, Елена Чернышова]",... публиковать данные о смертности от корон...,Ведомство объяснило это решение «очевидным сни...,Росстат перестал публиковать данные о смертнос...
1,634041959a794762778cad5a,https://www.rbc.ru/crypto/news/634041959a79476...,1665158925,"Fri, 07 Oct 2022 19:08:45 +0300",BNB Chain решит судьбу заблокированных криптоа...,{'url': 'https://s0.rbk.ru/v6_top_pics/resized...,Крипто,None,None,[Анастасия Кузьмичева],... Больше новостей о криптовалютах вы найде...,На повестку будут вынесены вопросы о заморозке...,"Получайте рассылку с новостями, которые касают..."
2,63404f449a794768137fd32d,https://www.rbc.ru/politics/07/10/2022/63404f4...,1665159151,"Fri, 07 Oct 2022 19:12:31 +0300",Как лидеры СНГ приехали на встречу в Санкт-Пет...,{'url': ''},None,Политика,None,None,В Петербурге прошел неформальный саммит лидер...,В Петербурге прошел неформальный саммит лидеро...,
3,6340315b9a79475c585c07f3,https://www.rbc.ru/politics/07/10/2022/6340315...,1665159391,"Fri, 07 Oct 2022 19:16:31 +0300",СПЧ ООН поддержал назначение спецдокладчика по...,{'url': 'https://s0.rbk.ru/v6_top_pics/resized...,None,Политика,None,[Елена Чернышова],... и Генеральной ассамблее на ее 78-й сесси...,Совет ООН по правам человека принял проект рез...,Совет ООН по правам человека на 51-й сессии в ...
4,63404d3c9a794766cf75f120,https://nn.rbc.ru/nn/freenews/63404d3c9a794766...,1665159529,"Fri, 07 Oct 2022 19:18:49 +0300",Сервисом «Стать добровольцем» воспользовались ...,{'url': 'https://s0.rbk.ru/v6_top_pics/resized...,None,None,None,None,"... оформлении заявки не возникает», — подче...",None,Сервисом «Стать добровольцем» на Госуслугах во...


### Лента.ру

* __query__ - поисковой запрос (ключевое слово)

* __offset__ - cмещение поисковой выдачи (от 0 до __size__)

* __size__ - количество статей. Ограничено время запроса, точного лимита нет. 1000 работает почти всегда 

* __sort__ - сортировка по дате: (2) - по убыванию, (3) - по возрастанию; по релевантности (1) 

* __title_only__ - точная фраза в заголовке (1)

* __domain__ - ? 

* __material__ - материалы: Все материалы (0). Новость (1). ["0", "1", "2", "3", "4", ...]

* __block__ - рубрика: Экономика (4). Все рубрики (0). ["0", "1", "2", "3", "4", ...]

* __dateFrom__ - с даты

* __dateTo__ - по дату

_Чтобы не специфировать параметр, оставляем поле пустым_

In [13]:
# Задаем тут параметры
use_parser = "LentaRu"

query = ''
offset = 0
size = 1000
sort = "3"
title_only = "0"
domain = "1"
material = "0"
bloc = "4"
dateFrom = '2020-01-01'
dateTo = "2020-03-31"

if use_parser == "LentaRu":
    param_dict = {'query'     : query, 
                  'from'      : str(offset),
                  'size'      : str(size),
                  'dateFrom'  : dateFrom,
                  'dateTo'    : dateTo,
                  'sort'      : sort,
                  'title_only': title_only,
                  'type'      : material, 
                  'bloc'      : bloc,
                  'domain'    : domain}

print(use_parser, "- param_dict:", param_dict)

LentaRu - param_dict: {'query': '', 'from': '0', 'size': '1000', 'dateFrom': '2020-01-01', 'dateTo': '2020-03-31', 'sort': '3', 'title_only': '0', 'type': '0', 'bloc': '4', 'domain': '1'}


In [14]:
# Тоже будем собирать итеративно, правда можно ставить time_step побольше, т.к.
# больше лимит на запрос статей. И Работает быстрее :)
assert use_parser == "LentaRu"
parser = lentaRu_parser()
tbl = parser.get_articles(param_dict=param_dict,
                         time_step = 37,
                         save_every = 5, 
                         save_excel = True)
print(len(tbl.index))
tbl.head()

Parsing articles from 2020-01-01 to 2020-02-07


/tmp/ipykernel_5304/3409529851.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 2020-02-08 to 2020-03-16


/tmp/ipykernel_5304/3409529851.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Parsing articles from 2020-03-17 to 2020-03-31


/tmp/ipykernel_5304/3409529851.py:89: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out = out.append(self._get_search_table(param_copy), ignore_index=True)


Finish
300


,docid,url,title,modified,lastmodtime,type,domain,status,part,bloc,tags,image_url,pubdate,text,rightcol,snippet
0,943747,https://lenta.ru/news/2020/01/01/org/,В России утвердили стандарты органической еды,1577838649,1577838649,1,1,0,0,4,"[7, 8]",https://icdn.lenta.ru/images/2019/12/31/13/201...,1577838649,Фото: Rupert Oberhäuser / Globallookpress.com ...,Вступил в силу принятый в 2018 году закон,Фото: Rupert Oberhäuser / ... для производител...
1,943634,https://lenta.ru/news/2020/01/01/theaters/,Российские театры освободили от налогов,1577840882,1577840883,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/30/20/201...,1577840882,Фото: Константин Кокошкин / «Коммерсантъ» Росс...,Обнулить базу удастся при условии строго следо...,Фото: Константин Кокошкин / «Коммерсантъ»... п...
2,943811,https://lenta.ru/news/2020/01/01/tariff/,Покупать в интернете за границей стало сложнее,1577847420,1577958238,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/17/201...,1577847420,Фото: Depositphotos Покупать в интернете за гр...,Порог беспошлинного ввоза снижен до 200 евро з...,"Фото: Depositphotos Покупать в интернете ..., ..."
3,943816,https://lenta.ru/news/2020/01/01/gas/,На Украине подорожал газ,1577850424,1577850425,1,1,0,0,4,"[7, 8, 198]",https://icdn.lenta.ru/images/2019/12/31/18/201...,1577850424,Фото: Sean Gallup / Getty Images Цена на газ д...,Стоимость тысячи кубометров для населения дост...,"Фото: Sean Gallup / Getty Images Цена на ...,3..."
4,943729,https://lenta.ru/news/2020/01/01/self/,В России стало больше самозанятых,1577854128,1577854128,1,1,0,0,4,[7],https://icdn.lenta.ru/images/2019/12/31/11/201...,1577854128,Фото: Олег Харсеев / «Коммерсантъ» В России ст...,Режим распространили на 19 новых регионов,Фото: Олег Харсеев / «Коммерсантъ» В ... распр...


In [ ]:
core-date-format date

In [ ]:
<div class="article__bar first">\n                <div class="article__status">\n                                        <span class="status__block">\n                        <core-date-format date="

In [35]:
rq.get('https://www.klerk.ru/buh/news/537015/').text

'<!DOCTYPE HTML>\n<html lang="ru" xmlns:fb="http://ogp.me/ns/fb#" xmlns:og="http://ogp.me/ns#">\n<head>\n    <title>Добровольные страховые взносы самозанятых будут снижаться, если бизнесмен не ходит на больничный</title>\n    <link title="Клерк.ру — практическая помощь бухгалтеру" type="application/rss+xml" rel="alternate"\n          href="https://www.klerk.ru/xml/index.xml"/>\n    <link rel="search" href="/service/opensearch.xml" title="Клерк.Ру" type="application/opensearchdescription+xml"/>\n    <link rel="image_src" href="//www.klerk.ru/img/og/clean_site.png"/>\n            <link rel="canonical" href="/buh/news/537015/" />        <meta charset="utf-8"/>\n    <!--[if IE]>\n    <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1"/><![endif]-->\n    <meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=no"/>\n\n    <meta name="description"\n          content="Минтруд разработал параметры добровольного страхования самозанятых на

In [21]:
from datetime import datetime

In [22]:
datetime.time(1665174708)

TypeError: descriptor 'time' for 'datetime.datetime' objects doesn't apply to a 'int' object

In [23]:
import time

In [ ]:
pip3 

In [29]:
datetime.fromtimestamp(1665175075/1000.0)

datetime.datetime(1970, 1, 20, 6, 32, 55, 75000)